# ReAct
- Reasoning + Acting을 의미하며, LLM의 reasoning 능력과 descision-making process 능력을 향상시키는 prompting 기술이다

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "ReAct Tutorial"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
tools = [TavilySearchResults(max_results=1)]

In [4]:
prompt = hub.pull("hwchase17/react")

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [16]:
llm = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model = "xionic-ko-llama-3-70b",
)

agent = create_react_agent(llm, tools, prompt)


In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({"input": "What is Langchain?"})



> Entering new AgentExecutor chain...
 I need to find out what Langchain is.
Action: tavily_search_results_json
Action Input: "What is Langchain?"[{'url': 'https://python.langchain.com/v0.2/docs/introduction/', 'content': "Introduction. LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle: Development: Build your applications using LangChain's open-source building blocks and components. Hit the ground running using third-party integrations and Templates."}] Langchain is a framework for developing applications powered by large language models (LLMs).
Final Answer: Langchain is a framework for developing applications powered by large language models (LLMs).

> Finished chain.


{'input': 'What is Langchain?',
 'output': 'Langchain is a framework for developing applications powered by large language models (LLMs).'}

In [15]:
prompt = hub.pull("hwchase17/react-chat")

In [17]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Your name is Bob
```

> Finished chain.


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
 'chat_history': 'Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you',
 'output': 'Your name is Bob\n```'}